<a href="https://colab.research.google.com/github/Zolinad/Topic_Redes_Neurais/blob/main/CNN%20com%20dataset%20MNIST/AV3_modelo_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REDES NEURAIS CONVOLUCIONAIS (CNN)
**AVALIAÇÃO 3**: Seguir as instruções no arquivo doc no SIGAA

## **Código 1 (Mostrar imagens do dataset MNIST)**

In [ ]:
from keras.datasets import mnist
import matplotlib.pyplot as plt

# Lista de nomes para cada classe do dataset MNIST (dígitos de 0 a 9)
mnist_class_names = {
    0: "Zero",
    1: "Um",
    2: "Dois",
    3: "Três",
    4: "Quatro",
    5: "Cinco",
    6: "Seis",
    7: "Sete",
    8: "Oito",
    9: "Nove"
}

# Carregar o dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Varredura em cada imagem no dataset
for i in range(30):  # mostrando apenas 30 imagens do dataset
    # Pegar uma imagem do dataset
    sample_image = x_train[i]
    # Pegar o id da classe associada à imagem
    image_class_number = y_train[i]
    # Procurar o nome da classe baseado no id da classe
    image_class_name = mnist_class_names[image_class_number]

    # Plotar a imagem
    plt.imshow(sample_image, cmap='gray')  # Usar cmap='gray' para exibir em escala de cinza
    # Rotular a imagem
    plt.title(image_class_name)
    # Mostrar o plot na tela
    plt.show()


---


## **Código 2 (TREINAR uma CNN usando o dataset MNIST)**

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

# O dataset MNIST possui 70.000 imagens de números, de 0 a 9.

# Carregar o dataset MNIST nos conjuntos de TREINO (60.000)  e de TESTE (10.000)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Ajustar as dimensões do dataset para (28, 28, 1) e normalizar os valores
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')
x_train /= 255
x_test /= 255

# Converter os rótulos de classe para vetores one-hot encoded
# (necessário para classificação multiclasse)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


# Criar o modelo de CNN e adicionar as camadas
model = Sequential()


# Camada de ENTRADA explícita, que define o formato da entrada (28x28 e 1 canal)
camada_entrada = (ALTURA, LARGURA, Q_CANAL)


# DUAS Camadas de convolução com 32 filtros (kernels), com tamanho de kernel 3x3
model.add(Conv2D(32, (K, K), padding='same', input_shape = camada_entrada, activation="relu"))
model.add(Dropout(0.25))  # dropout deve ser antes do POOLING
model.add(Conv2D(32, (K, K), activation="relu"))
model.add(Dropout(0.25))  # dropout deve ser antes do POOLING
# UMA Camada de Pooling, com matrizes de tamanho 2x2 (janelas de pooling)
model.add(MaxPooling2D(pool_size=(P, P)))


# + DUAS camadas de Convolução com 64 kernels de tamanho 3x3
model.add(Conv2D(64, (K, K), padding='same', activation="relu"))
model.add(Dropout(0.25))  # dropout deve ser antes do POOLING
model.add(Conv2D(64, (K, K), activation="relu"))
model.add(Dropout(0.25))  # dropout deve ser antes do POOLING
# + UMA camada de Pooling, com matrizes de tamanho 2x2
model.add(MaxPooling2D(pool_size=(P, P)))


# Camada de Flattening (achatamento das matrizes)
model.add(Flatten())
# Exibir o resumo do modelo
model_summary = model.summary()
# Quatidade de neurônios na camada densa (de acordo com a saída na camada de achatamento (Flatten))
camada_densa = int(input("Digite a quantidade de neurônios na camada densa: "))


# Deep Learning - Camadas da Rede Neural densa (fully conected)
model.add(Dense(camada_densa, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(camada_densa, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(camada_densa, activation="relu"))
model.add(Dropout(0.5))


# Camada de saida com 10 classes (de 0 a 9)
model.add(Dense(10, activation="softmax"))
model_summary = model.summary()


# Compilar o modelo de uma CNN
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)


# Treinamento do modelo de uma CNN
model.fit(
    x_train,
    y_train,
    batch_size = 200,  # Aumentando o batch-size reduz-se o tempo de treinamento
    epochs = 20,  # Menos épocas evitam overfitting e reduzem o tempo de treinamento
    validation_data = (x_test, y_test),
    shuffle=True
)

# Salvar a estrutura do modelo da rede neural
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Salvar os pesos da rede neural
model.save_weights("model.weights.h5")


---

## **Código 3 (CLASSIFICAR os números contidos nas imagens de teste)**

In [ ]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
import numpy as np
import matplotlib.pyplot as plt

# As classes do dataset MNIST são simplesmente os dígitos de 0 a 9
rotulos_das_classes = {
    0: "Zero",
    1: "Um",
    2: "Dois",
    3: "Três",
    4: "Quatro",
    5: "Cinco",
    6: "Seis",
    7: "Sete",
    8: "Oito",
    9: "Nove"
}

# Carregar o arquivo json que contém o modelo estruturado da rede neural
f = Path("model_structure.json")
model_structure = f.read_text()

# Recriar o modelo da rede neural
model = model_from_json(model_structure)

# Recarregar os pesos treinados do modelo de rede neural
model.load_weights("model.weights.h5")

# Carregar o dataset MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Carregar uma imagem para teste, redimensionando-a para 28x28 pixels,
# como requerido no modelo MNIST.
img = image.load_img("imagem_de_teste.jpg", color_mode="grayscale", target_size=(ALTURA, LARGURA))

# Converter a imagem para um array numpy e normalizar os valores (0 a 1)
image_to_test = image.img_to_array(img) / 255.0

# Mostrar a imagem que será testada
plt.imshow(img, cmap='gray')
plt.show()

# Adicionar uma quarta dimensão à imagem (para representar o batch size, que é esperado pelo Keras)
lista_de_imagens = np.expand_dims(image_to_test, axis=0)

# Fazer a predição usando o modelo
resultado = model.predict(lista_de_imagens)

# Já que estamos testando apenas uma imagem, só precisamos checar o primeiro resultado
resultado_unico = resultado[0]

# Mostrar a probabilidade para todas as classes
print("Probabilidades para cada classe:")
for i, prob in enumerate(resultado_unico):
    print(f"{rotulos_das_classes[i]}: {prob:.2f}")

# Vamos calcular a classe mais provável.
indice_da_classe_mais_provavel = int(np.argmax(resultado_unico))
probabilidade_da_classe = resultado_unico[indice_da_classe_mais_provavel]

# Pegar o nome da classe mais provável (o dígito)
classe = rotulos_das_classes[indice_da_classe_mais_provavel]

# Mostrar o resultado da classe mais provável
print(f"\nEsta imagem mostra o dígito {classe} - Probabilidade: {probabilidade_da_classe:.2f}")
